In [2]:
import os
import pandas as pd
import numpy as np
from pybaseball import batting_stats

KeyboardInterrupt: 

In [ ]:
START = 2002
END = 2022

In [ ]:
batting = batting_stats(START, END, qual=200)

KeyboardInterrupt: 

In [ ]:
batting.to_csv("batting.csv")

NameError: name 'batting' is not defined

In [3]:
batting = pd.read_csv('batting.csv')

In [4]:
grouped_batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1)

In [5]:
grouped_batting

,Unnamed: 0,IDfg,Season,Name,Team,Age,G,AB,PA,H,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,...,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
1,1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,...,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
2,8,15640,2022,Aaron Judge,NYY,30,157,570,696,177,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.4
3,2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,...,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2
4,15,13611,2018,Mookie Betts,BOS,25,136,520,614,180,...,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7109,7042,9272,2018,Chris Davis,BAL,32,128,470,522,79,...,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN,-2.6
7110,6535,45,2012,Rod Barajas,PIT,36,104,321,361,66,...,NaN,0.0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.6
7111,6673,319,2011,Adam Dunn,CHW,31,122,415,496,66,...,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9
7112,6988,620,2002,Neifi Perez,KCR,29,145,554,585,131,...,NaN,NaN,NaN,0,0.130,0.187,NaN,NaN,NaN,-2.9


In [6]:

# move next season war to row in prev season to use as labels for ML
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    
    return player

batting_with_next_war = grouped_batting.groupby("IDfg", group_keys=False).apply(next_season)

/var/folders/z5/81gv5lc94lg2dbfrv548wwr80000gn/T/ipykernel_9880/3038278376.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting_with_next_war = grouped_batting.groupby("IDfg", group_keys=False).apply(next_season)


In [7]:
batting_with_next_war[["Name", "Season", "WAR", "Next_WAR"]]

,Name,Season,WAR,Next_WAR
3946,Alfredo Amezaga,2006,1.1,2.0
2607,Alfredo Amezaga,2007,2.0,1.2
3787,Alfredo Amezaga,2008,1.2,NaN
1020,Garret Anderson,2002,3.7,5.1
424,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
4667,Owen Miller,2022,0.7,NaN
6108,Andrew Vaughn,2021,-0.2,-0.5
6450,Andrew Vaughn,2022,-0.5,NaN
5238,Ha-seong Kim,2021,0.4,3.6


In [8]:
null_count = batting_with_next_war.isnull().sum()
null_count, batting_with_next_war.columns

(Unnamed: 0       0
 IDfg             0
 Season           0
 Name             0
 Team             0
               ... 
 xBA           6754
 xSLG          6754
 xwOBA         6754
 L-WAR            0
 Next_WAR      1179
 Length: 322, dtype: int64,
 Index(['Unnamed: 0', 'IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA',
        'H',
        ...
        'HardHit', 'HardHit%', 'Events', 'CStr%', 'CSW%', 'xBA', 'xSLG',
        'xwOBA', 'L-WAR', 'Next_WAR'],
       dtype='object', length=322))

In [11]:
complete_cols = list(batting_with_next_war.columns[null_count == 0])
complete_cols

['Unnamed: 0',
 'IDfg',
 'Season',
 'Name',
 'Team',
 'Age',
 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'GDP',
 'SB',
 'CS',
 'AVG',
 'GB',
 'FB',
 'LD',
 'IFFB',
 'Pitches',
 'Balls',
 'Strikes',
 'IFH',
 'BU',
 'BUH',
 'BB%',
 'K%',
 'BB/K',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'wOBA',
 'wRAA',
 'wRC',
 'Bat',
 'Rep',
 'Pos',
 'RAR',
 'WAR',
 'Dol',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'PH',
 'WPA/LI',
 'Clutch',
 'FB% (Pitch)',
 'FBv',
 'SL%',
 'SLv',
 'CB%',
 'CBv',
 'CH%',
 'CHv',
 'wFB',
 'wSL',
 'wCB',
 'wCH',
 'wFB/C',
 'wSL/C',
 'wCB/C',
 'wCH/C',
 'O-Swing%',
 'Z-Swing%',
 'Swing%',
 'O-Contact%',
 'Z-Contact%',
 'Contact%',
 'Zone%',
 'F-Strike%',
 'SwStr%',
 'BsR',
 'Def',
 'wSB',
 'UBR',
 'Age Rng',
 'Off',
 'Lg',
 'wGDP',
 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%',
 'TTO%',
 'AVG+

In [13]:
batting = batting_with_next_war[complete_cols + ["Next_WAR"]].copy()
batting.dtypes

Unnamed: 0      int64
IDfg            int64
Season          int64
Name           object
Team           object
               ...   
Events          int64
CStr%         float64
CSW%          float64
L-WAR         float64
Next_WAR      float64
Length: 134, dtype: object

In [14]:
batting.dtypes[batting.dtypes == "object"]

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [17]:
del batting["Dol"]

In [18]:
batting["Age Rng"]

3946    28 - 28
2607    29 - 29
3787    30 - 30
1020    30 - 30
424     31 - 31
         ...   
4667    25 - 25
6108    23 - 23
6450    24 - 24
5238    25 - 25
1128    26 - 26
Name: Age Rng, Length: 6754, dtype: object

In [19]:
del batting["Age Rng"]

In [27]:
batting["team_codes"] = batting["Team"].astype("category").cat.codes

In [32]:
batting_full = batting.copy()
batting = batting.dropna().copy()

In [30]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = Ridge(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=20, direction="forward", cv=split,n_jobs=4)

In [42]:
removed_columns = ["Next_WAR", "Name", "Team", "IDfg", "Season"]
selected_columns = batting.columns[~batting.columns.isin(removed_columns)]

In [44]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])

In [46]:
batting.describe()

,Unnamed: 0,IDfg,Season,Age,G,AB,PA,H,1B,2B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR,team_codes
count,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,...,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000
mean,0.451542,5366.785830,2011.163229,0.360600,0.652755,0.478666,0.480943,0.365973,0.290481,0.399279,...,0.403164,0.410923,0.511026,0.478646,0.172991,0.498932,0.545898,0.322041,1.792969,0.474128
std,0.279450,5133.255295,5.612014,0.147476,0.255929,0.242481,0.262290,0.182585,0.138786,0.171732,...,0.131213,0.121082,0.130359,0.133992,0.273858,0.137180,0.120701,0.122148,1.980831,0.305105
min,0.000000,1.000000,2002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.100000,0.000000
25%,0.209265,1131.500000,2006.000000,0.269231,0.478632,0.275180,0.257785,0.211207,0.179245,0.258621,...,0.315789,0.331461,0.420290,0.387755,0.000000,0.408511,0.466960,0.234177,0.400000,0.205882
50%,0.431885,3531.000000,2011.000000,0.346154,0.709402,0.505396,0.508651,0.370690,0.283019,0.379310,...,0.398496,0.404494,0.507246,0.489796,0.000000,0.493617,0.546256,0.303797,1.500000,0.470588
75%,0.681358,9015.000000,2016.000000,0.461538,0.871795,0.688849,0.710208,0.508621,0.391509,0.517241,...,0.488722,0.483146,0.594203,0.564626,0.346411,0.591489,0.625551,0.392405,2.900000,0.735294
max,1.000000,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


In [47]:
sfs.fit(batting[selected_columns], batting["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [50]:
predictors = list(selected_columns[sfs.get_support()])

In [ ]:
def backtest(data, model, predictors, start=5,step=1):
    ...